In [51]:
import copy
import random
import itertools
from tabulate import tabulate
import json

def sort_subsublists(lst):
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            lst[i][j].sort()
    return lst

def setStar(k,m):
    config = []
    central = []
    for chip in range(1,k*m+1):
        central.append(chip)
    for branch in range(k):
        branch_list = []
        for vertex in range(m+2):
            branch_list.append([])
        branch_list[0]=central
        config.append(branch_list)
    return config

def fireCentralSp(og_config,og_chips):
    config = copy.deepcopy(og_config)
    chipsFired = copy.deepcopy(og_chips)
    k = len(config)
    if len(config[0][0]) >= k:
        for chip in chipsFired:
            config[0][0].remove(chip)
        for i in range(k):
            chip = min(chipsFired)
            config[i][1].append(chip)
            chipsFired.remove(chip)
        config = sort_subsublists(config)
        return config
    else:
        return None

def fireBranchSp(og_config,b,v,og_chips):
    config = copy.deepcopy(og_config)
    chipsFired = copy.deepcopy(og_chips)
    if len(config[b][v]) >= 2:
        config[b][v-1].append(min(chipsFired))
        config[b][v+1].append(max(chipsFired))
        config[b][v].remove(min(chipsFired))
        config[b][v].remove(max(chipsFired))
        config = sort_subsublists(config)
        return config
    else:
        return None

def subsets_of_k(nested_list, k):
    all_subsets = []
    
    for sublist in nested_list:
        if len(sublist) >= k:
            subsets = list(itertools.combinations(sublist, k))
            all_subsets.extend(subsets)
    
    return all_subsets

def all_pairs(lst):
    pairs = []
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            pairs.append((lst[i], lst[j]))
    return pairs

def all_tuples(lst, k):
    return list(itertools.combinations(lst, k))

def getFirableSp(og_config):
    config = copy.deepcopy(og_config)
    k = len(config)
    l = len(config[0])
    firable = []
    firable_at_zero = all_tuples(config[0][0],k)
    for ktup in firable_at_zero:
        if fireCentralSp(config,list(ktup)) is not None:
            firable.append([0,0,list(ktup)])
    for b in range(k):
        for v in range(1,l):
            firable_on_branch = all_pairs(config[b][v])
            for pair in firable_on_branch:
                if fireBranchSp(config,b,v,list(pair)) is not None:
                    firable.append([b,v,list(pair)])
    return firable

def generate_reachable_configs(config):
    reachableConfigs = [config]
    new_configs_found = True
    while new_configs_found:
        new_configs_found = False
        for current_config in reachableConfigs[:]:
            firable = getFirableSp(current_config)
            for triple in firable:
                if triple[1] == 0:
                    new_config = fireCentralSp(current_config,triple[2])
                    if new_config is not None and new_config not in reachableConfigs:
                        reachableConfigs.append(new_config)
                        new_configs_found = True
                else:
                    new_config = fireBranchSp(current_config,triple[0],triple[1],triple[2])
                    if new_config is not None and new_config not in reachableConfigs:
                        reachableConfigs.append(new_config)
                        new_configs_found = True
    return reachableConfigs

def genEdges(config):
    reachableConfigs = generate_reachable_configs(config)
    numVertices = len(reachableConfigs)
    edges = []
    for i in range(numVertices):
        reached = reachableConfigs[i]
        check_firings = getFirableSp(reached)
        for firable in check_firings:
            if firable[1] == 0:
                firedConfig = fireCentralSp(reached,firable[2])
            else:
                firedConfig = fireBranchSp(reached,firable[0],firable[1],firable[2])
            if firedConfig in reachableConfigs and [i,reachableConfigs.index(firedConfig)] not in edges:
                edges.append([i,reachableConfigs.index(firedConfig)])
            else:
                pass
    return edges

# def vertex_prob_label(digraph):
#     digraph.set_vertex(0,1)
#     levels = len(digraph.longest_path())
#     for lvl in range(1,levels):
#         lvl_vertices = [v for v in digraph.vertices() if digraph.shortest_path_length(0,v) == lvl]
#         for v in lvl_vertices:
#             new_label = sum([digraph.get_vertex(n)/digraph.out_degree(n) for n in digraph.neighbors_in(v)])
#             digraph.set_vertex(v,new_label)
#         level_sum = sum([digraph.get_vertex(v) for v in lvl_vertices])
#         if level_sum == 1:
#             pass
#         else:
#             print(f"level {lvl} has an error: probabilities sum to {level_sum}")
#     return digraph

# PROMPT
def flatten_list_of_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def checkSort(config):
    branches = []
    is_sorted = True
    for sublst in config:
        flt_sublst = flatten_list_of_lists(sublst)
        branches.append(flt_sublst)
        flt_srt_sublst = sorted(flt_sublst)
        if flt_srt_sublst != flt_sublst:
            # print(f"not sorted along branch {final.index(sublst)}")
            is_sorted = False
            break
    for v in range(len(branches[0])):
        level = []
        for branch in branches:
            level.append(branch[v])
        level_sorted = sorted(level)
        if level_sorted != level:
            # print(f"{branches} not sorted around ring {v+1}")
            is_sorted = False
            break
    return is_sorted

In [52]:
def optionsTestStar(k,m,print_poset,print_rank,print_num_terms,print_Lkm,print_all_or_terminal,check_sort,
                    print_paths,print_levels,print_shadows,print_PQ,table_format):
    # checking input validity
    TFList = [True,False]
    if print_poset not in TFList or print_paths not in TFList or print_levels not in TFList or print_shadows not in TFList or check_sort not in TFList:
        # or print_probabilities not in TFList
        print(f"most inputs should all be either True or False (without quotation marks)")
        return None
    if print_all_or_terminal not in ['all', 'terminal', 'none']:
        print(f"choose print_all_or_terminal to be 'all' or 'terminal' or 'none' (with quotations) instead of {print_all_or_terminal}")
        return None
    if table_format not in ['outline', 'latex', 'github']:
        print(f"table_format should be either 'outline' or 'latex' or 'github' (with quotations)")
        return None

    # setup

    # for nice copy & pasting
    if table_format == 'latex':
        line_end = '\\\\'
    else:
        line_end = ' '
        
    print(f"testing (k,m)=({k},{m}) {line_end}")
    config = setStar(k,m)
    reachables = generate_reachable_configs(config)
    edges = genEdges(config)
    digraph = DiGraph(edges)
    poset = Poset(digraph)

    # print_poset
    if print_poset == True:
        poset.show()
        print(" ")

    # print_rank, print_num_terms, print_Lkm
    rnk = poset.rank()
    Lkm = len(poset.maximal_chains())
    max_elts = poset.maximal_elements()
    if print_rank == True:
        print(f"rank: {rnk} {line_end}")
    if print_num_terms == True:
        print(f"unique terminal configurations: {len(max_elts)} {line_end}")
    if print_Lkm == True:
        print(f"L({k},{m}) = {Lkm} {line_end}")
    
    elements_to_print = []
    # print_all_or_terminal
    if print_all_or_terminal == 'all':
        for elt in poset:
            elements_to_print.append(elt)
    elif print_all_or_terminal == 'terminal':
        elements_to_print = max_elts
    else:
        return None

    table = []
    heading = {
        'index': 'index',
        'name': 'element',
    }

    for elt_index in elements_to_print:
        dictionary = {}
        dictionary['index'] = elt_index
        dictionary['name'] = reachables[elt_index]
        table.append(copy.deepcopy(dictionary))

    # check_sort
    if check_sort == True:
        heading['sorted'] = 'sorted'
        for dictionary in table:
            dictionary['sorted'] = checkSort(dictionary['name'])

    # # print_probabilities
    # if print_probabilities == True:
    #     heading['probability'] = 'probability'
    #     labeled_digraph = vertex_prob_label(digraph)
    #     for v in digraph.vertices():
    #         for dictionary in table:
    #             if dictionary['index'] == v:
    #                 dictionary['probability'] = digraph.get_vertex(v)

    # print_paths
    if print_all_or_terminal == 'terminal' and print_paths == True:
        heading['paths'] = 'paths to configuration'
        for dictionary in table:
            dictionary['paths'] = 0
        for chain in poset.maximal_chains():
            for dictionary in table:
                if chain[rnk] == dictionary['index']:
                    dictionary['paths'] += 1

    # print_levels
    if print_all_or_terminal == 'all' and print_levels == True:
        heading['level'] = 'level'
        for vertex in table:
            vertex['level'] = digraph.shortest_path_length(0,vertex['index'])

    # print_shadows
    if print_all_or_terminal != 'none' and print_shadows == True:
        heading['shadow'] = 'chips per vertex'
        for dictionary in table:
            branch_shadows = []
            for sublist in dictionary['name']:
                vertex_shadows = []
                for subsublist in sublist:
                    vertex_shadows.append(len(subsublist))
                branch_shadows.append(vertex_shadows)
            dictionary['shadow'] = branch_shadows

    # flatten configurations to reduce space
    for dictionary in table:
        temp_lst = []
        for branch in dictionary['name']:
            new_branch = flatten_list_of_lists(branch)
            temp_lst.append(new_branch)
        dictionary['name'] = temp_lst
    
    # sort table by index
    table.sort(key=lambda x: x['index'])
    
    # if not printing the poset, don't print the indices of each config
    if print_poset == False:
        for dictionary in table:
            del dictionary['index']
        del heading['index']

    # print table
    print(tabulate(table, headers=heading, tablefmt=table_format))
    print(" ")

    # print_PQ
    if print_all_or_terminal == 'terminal' and print_paths == True and print_PQ == True:
        PQtable = []
        Qs = []
        for dictionary in table:
            if dictionary['paths'] not in Qs:
                Qs.append(dictionary['paths'])
        Qs.sort()
        for i in range(0,len(Qs)):
            # find P(k,m,i)
            Pi = 0
            for dictionary in table:
                if dictionary['paths'] == Qs[i]:
                    Pi += 1
            # append [i,P,Q] to PQtable
            PQtable.append([str(i+1),Pi,Qs[i]])
        print(tabulate(PQtable, headers=["i",f"P({k},{m},i)",f"Q({k},{m},i)"], tablefmt=table_format))
        print(" ")
    
    return None

"""
USAGE

k (positive integer)
    number of branches
    
m (positive integer)
    n=km is the total number of chips
    
print_poset (True OR False)

print_rank (True OR False)

print_num_terms (True OR False)
    prints number of terminal configurations

print_Lkm (True OR False)
    prints L(k,m)

print_all_or_terminal ('all' OR 'terminal' OR 'none')
    'all' prints a table of all reachable configurations
    'terminal' prints only stable configurations
    'none' only prints the rank, the number of unique terminal configurations, and L(k,m)

check_sort (True OR False)
    for each configuration, prints whether each configuration is sorted along both branches and rings
    
REMOVED print_probabilities (True OR False)
    prints the probability of each vertex being reached
    
print_paths (True OR False)
    prints the number of paths to a specific vertex
    only works if print_all_or_terminal == 'terminal'
    
print_levels (True OR False)
    prints the "level" of any element in the poset (initial being 0, terminals being the rank of the poset)
    only works if print_all_or_terminal == 'all'

print_shadows (True OR False)
    prints a list of lists telling you how many chips are on each vertex
    only works if print_all_or_terminal != 'none'

print_PQ (True OR False)
    prints a table of P(k,m,i) and Q(k,m,i)s
    only works if print_paths == True
    only works if print_all_or_terminal == 'terminal'
    
table_format ('outline' OR 'latex' OR 'github')
    'outline' prints the output in a readable table
    'latex' prints it in a LaTeX table format for copy & pasting
    'github' is nice for copy & pasting into Google Sheets and separating data to columns
"""

print("ready to go!")

ready to go!


In [53]:
%%time
# test

k=2
m=3
print_poset = False
print_rank = False
print_num_terms = False
print_Lkm = True
print_all_or_terminal = 'terminal'
check_sort = True
# print_probabilities = False
print_paths = True
print_levels = False
print_shadows = False
print_PQ = False
table_format = 'outline'

optionsTestStar(k,m,print_poset,print_rank,print_num_terms,print_Lkm,print_all_or_terminal,check_sort,print_paths,print_levels,print_shadows,print_PQ,table_format)

testing (k,m)=(2,3)  
L(2,3) = 181440  
+------------------------+----------+--------------------------+
| element                | sorted   |   paths to configuration |
+========================+==========+==========================+
| [[1, 3, 5], [2, 4, 6]] | True     |                     8568 |
| [[1, 2, 5], [3, 4, 6]] | True     |                    22680 |
| [[1, 2, 3], [4, 5, 6]] | True     |                    74088 |
| [[1, 3, 4], [2, 5, 6]] | True     |                    24696 |
| [[1, 2, 4], [3, 5, 6]] | True     |                    51408 |
+------------------------+----------+--------------------------+
 
CPU times: user 6.08 s, sys: 60.8 ms, total: 6.14 s
Wall time: 6.02 s


In [56]:
%%time

print_poset = False
print_rank = True
print_num_terms = False
print_Lkm = False
print_all_or_terminal = 'terminal'
check_sort = True
# print_probabilities = False
print_paths = True
print_levels = True
print_shadows = False
print_PQ = False
table_format = 'outline'

for pair in [(1,1),(1,2),(1,3),(2,1),(2,2),(3,1),(3,2)]:
    k = pair[0]
    m = pair[1]
    optionsTestStar(k,m,print_poset,print_rank,print_num_terms,print_Lkm,print_all_or_terminal,check_sort,print_paths,print_levels,print_shadows,print_PQ,table_format)

testing (k,m)=(1,1)  
rank: 1  
+-----------+----------+--------------------------+
| element   | sorted   |   paths to configuration |
+===========+==========+==========================+
| [[1]]     | True     |                        1 |
+-----------+----------+--------------------------+
 
testing (k,m)=(1,2)  
rank: 4  
+-----------+----------+--------------------------+
| element   | sorted   |   paths to configuration |
+===========+==========+==========================+
| [[1, 2]]  | True     |                        2 |
+-----------+----------+--------------------------+
 
testing (k,m)=(1,3)  
rank: 10  
+-------------+----------+--------------------------+
| element     | sorted   |   paths to configuration |
+=============+==========+==========================+
| [[1, 2, 3]] | True     |                       60 |
+-------------+----------+--------------------------+
 
testing (k,m)=(2,1)  
rank: 1  
+------------+----------+--------------------------+
| element    | sorted 

# Random stabilization

In [47]:
# This section is for running one "random" stabilization
# Note that it is not exactly random because it run (I think) with an equal probability of firing each firable vertex,
# and then from there it randomly chooses chips (of the appropriate amount) to fire
# We can probably change this but for now this gives a decent estimate

# PROMPT: write a sagemath code that will take a list of at least k items and randomly choose k of those items
def choose_k_random_items(items, k):
    # Ensure that the list has at least k items
    if len(items) < k:
        raise ValueError("The list must have at least k items")
    
    # Randomly choose k items from the list
    chosen_items = random.sample(items, k)
    return chosen_items

# PROMPT: write a sagemath function that sorts each subsublist in a list of lists of lists
def sort_subsublists(lst):
    """
    Sorts each subsublist in a list of lists of lists.
    
    Args:
    lst (list): A list of lists of lists.

    Returns:
    list: The sorted list of lists of lists.
    """
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            lst[i][j].sort()  # Sort each subsublist
    return lst

# setup:
# branches numbered 0 to k-1
# center vertex is the 0th vertex for all branches
# config = [ branch=[ vertex=[chips] ] ]
# ex if branch 1 vertex 2 has chip 7, 7 in config[1][2] == True
def setStar(k,m):
    config = []
    central = []
    for chip in range(1,k*m+1):
        central.append(chip)
    for branch in range(k):
        branch_list = []
        for vertex in range(m+2):
            branch_list.append([])
        branch_list[0]=central
        config.append(branch_list)
    return config

# to fire the center:
# if config[0][0] has at least k chips
# choose k chips
# send the smallest to config[0][1], next smallest to config[1][1], etc
# remove those k chips from config[b][0] for all b in [0,k-1]
def fireCentral(og_config):
    config = copy.deepcopy(og_config)
    k = len(config)
    if len(config[0][0]) >= k:
        chipsFired = choose_k_random_items(config[0][0], k)
        # print(f"firing chips {chipsFired}")
        for chip in chipsFired:
            config[0][0].remove(chip)
        for i in range(k):
            chip = min(chipsFired)
            config[i][1].append(chip)
            chipsFired.remove(chip)
        config = sort_subsublists(config)
        return config
    else:
        return None

# to fire vertex v of branch b (b in [0,k-1], v>=1):
# if config[b][v] has at least 2 chips
# choose two of those chips
# send the smaller to config[b][v-1] and the larger to config[b][v+1]
# remove those 2 chips from config[b][v]
def fireBranch(og_config,b,v):
    config = copy.deepcopy(og_config)
    if len(config[b][v]) >= 2:
        chipsFired = choose_k_random_items(config[b][v], 2)
        config[b][v-1].append(min(chipsFired))
        config[b][v+1].append(max(chipsFired))
        config[b][v].remove(min(chipsFired))
        config[b][v].remove(max(chipsFired))
        config = sort_subsublists(config)
        return config
    else:
        return None

# get firable vertices and fire a random one
def getFirable(og_config):
    config = copy.deepcopy(og_config)
    k = len(config)
    l = len(config[0])
    firable = []
    if fireCentral(config) is not None:
        firable.append([0,0])
    for b in range(k):
        for v in range(1,l):
            if fireBranch(config,b,v) is not None:
                firable.append([b,v])
    # print(f"firable: {firable}")
    return firable

def fireRandom(og_config):
    config = copy.deepcopy(og_config)
    firable = getFirable(config)
    if firable != []:
        firing = random.sample(firable, 1)[0]
        # print(f"firing (b,v): {firing}")
        if firing[1] == 0:
            output = fireCentral(config)
            return output
        else:
            # print(f"firing {config} at branch {firing[0]} vertex {firing[1]}")
            output = fireBranch(config,firing[0],firing[1])
            return output
    else:
        return None

# to play the game:
# initialize k branches of length m+2 (including center) and n=km chips at the central vertex
# fire a random vertex
# repeat until nothing can fire
# when nothing can fire:
    # print final configuration
def playGame(k,m):
    config = setStar(k,m)
    fireAgain = True
    current = config
    while fireAgain == True:
        fireAgain = False
        output = fireRandom(current)
        # print(f"fired to: {output}")
        if output is not None:
            fireAgain = True
            current = output
        else:
            fireAgain = False
            # print(f"Central vertex: {current[0][0]}")
            for b in range(k):
                current[b].pop(0)
            # for b in range(k):
                # print(f"Branch {b}: {current[b]}")
    return current

def checkSortPrint(config):
    branches = []
    is_sorted = True
    for sublst in config:
        flt_sublst = flatten_list_of_lists(sublst)
        branches.append(flt_sublst)
        flt_srt_sublst = sorted(flt_sublst)
        if flt_srt_sublst != flt_sublst:
            print(f"not sorted along branch {final.index(sublst)}")
    for v in range(len(branches[0])):
        level = []
        for branch in branches:
            level.append(branch[v])
        level_sorted = sorted(level)
        if level_sorted != level:
            print(f"{branches} not sorted around ring {v+1}")
            is_sorted = False
    return is_sorted


# this plays the game but only prints something if the final config you get is unsorted
def playGameSort(k,m):
    final = playGame(k,m)
    is_sorted = checkSortPrint(final)
    return is_sorted

In [48]:
%%time

run_times = 1000
count = 0
for i in range(run_times+1):
    if playGameSort(3,4) == False:
        count += 1
print(" ")
print(f"{count}/{run_times} results not sorted")
print(" ")

[[1, 4, 5, 7], [2, 3, 9, 11], [6, 8, 10, 12]] not sorted around ring 2
[[1, 4, 5, 6], [2, 3, 8, 9], [7, 10, 11, 12]] not sorted around ring 2
[[1, 3, 6, 8], [2, 4, 5, 10], [7, 9, 11, 12]] not sorted around ring 3
[[1, 4, 5, 7], [2, 3, 9, 10], [6, 8, 11, 12]] not sorted around ring 2
[[1, 3, 7, 8], [2, 4, 5, 10], [6, 9, 11, 12]] not sorted around ring 3
[[1, 5, 6, 7], [2, 3, 8, 9], [4, 10, 11, 12]] not sorted around ring 2
[[1, 4, 5, 8], [2, 3, 7, 9], [6, 10, 11, 12]] not sorted around ring 2
[[1, 2, 6, 7], [3, 4, 5, 10], [8, 9, 11, 12]] not sorted around ring 3
[[1, 2, 6, 7], [3, 4, 5, 9], [8, 10, 11, 12]] not sorted around ring 3
[[1, 2, 7, 8], [3, 4, 5, 10], [6, 9, 11, 12]] not sorted around ring 3
[[1, 4, 5, 6], [2, 3, 9, 10], [7, 8, 11, 12]] not sorted around ring 2
[[1, 4, 5, 6], [2, 3, 9, 10], [7, 8, 11, 12]] not sorted around ring 2
[[1, 4, 5, 7], [2, 3, 8, 9], [6, 10, 11, 12]] not sorted around ring 2
[[1, 2, 5, 6], [3, 8, 9, 10], [4, 7, 11, 12]] not sorted around ring 2
[[1, 2